In [1]:
library(FigR)
library(Seurat)
library(genomation)
library(GenomicRanges)
library(BSgenome.Hsapiens.UCSC.hg38)
library(foreach)

Loading required package: Matrix

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2, rowMedians, rowMins,
    rowOrderStats,

Specify file path

In [2]:
path.pairs.E2G = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/240508/sc-E2G/test/results_K562_Xu/K562/Kendall/Pairs.tsv.gz"
path.matrix.atac_count = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/240508/sc-E2G/test/results_K562_Xu/K562/Kendall/atac_matrix.csv.gz"
path.matrix.rna_count = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/K562_Xu/1.prepare_data/1.seurat_pipeline.240507/rna_count_matrix.csv.gz"
dir.output = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/K562_Xu/3.Genome_wide_prediction/FigR/FigR.240517/"

Import candidate E-G pairs

In [3]:
pairs.E2G = readGeneric(path.pairs.E2G,
                        header = T,
                        keep.all.metadata = T)

Import ATAC matrix

In [4]:
matrix.atac_count = read.csv(path.matrix.atac_count,
                             row.names = 1,
                             check.names = F)
matrix.atac_count = Matrix(as.matrix(matrix.atac_count), sparse = TRUE)
matrix.atac = centerCounts(matrix.atac_count)

Matrix object input detectedCentering counts for cells sequentially in groups of size  1000  ..

Computing centered counts for cells:  1  to  1000 ..
Computing centered counts per cell using mean reads in features ..

Computing centered counts for cells:  1001  to  2000 ..
Computing centered counts per cell using mean reads in features ..

Computing centered counts for cells:  2001  to  3000 ..
Computing centered counts per cell using mean reads in features ..

Computing centered counts for cells:  3001  to  4000 ..
Computing centered counts per cell using mean reads in features ..

Computing centered counts for cells:  4001  to  5000 ..
Computing centered counts per cell using mean reads in features ..

Computing centered counts for cells:  5001  to  6000 ..
Computing centered counts per cell using mean reads in features ..

Computing centered counts for cells:  6001  to  7000 ..
Computing centered counts per cell using mean reads in features ..

Computing centered counts for cells:  

Import RNA matrix

In [5]:
matrix.rna_count = read.csv(path.matrix.rna_count,
                            row.names = 1,
                            check.names = F)
matrix.rna_count = Matrix(as.matrix(matrix.rna_count), sparse = TRUE)
matrix.rna_count = matrix.rna_count[,colnames(matrix.atac)]
matrix.rna = matrix.rna_count[rowSums(matrix.rna_count) > 0,]
matrix.rna = NormalizeData(matrix.rna_count)
rm(matrix.rna_count)

In [6]:
pairs.E2G.filter = pairs.E2G[pairs.E2G$TargetGene %in% rownames(matrix.rna) &
                             pairs.E2G$PeakName %in% rownames(matrix.atac)]

Prepare FigR input data

In [7]:
bed.peak = pairs.E2G
mcols(bed.peak) = NULL
bed.peak = unique(bed.peak)
bed.peak$PeakName = paste(seqnames(bed.peak),
                          start(bed.peak),
                          end(bed.peak),
                          sep = "-")
names(bed.peak) = bed.peak$PeakName
bed.peak = bed.peak[rownames(matrix.atac)]

In [8]:
ATAC.se = SummarizedExperiment(assays = SimpleList(counts = matrix.atac_count),
                               rowRanges = bed.peak)
rm(matrix.atac_count)
ATAC.se <- chromVAR::addGCBias(ATAC.se, genome = BSgenome.Hsapiens.UCSC.hg38)
ATAC.se

class: RangedSummarizedExperiment 
dim: 157600 7821 
metadata(0):
assays(1): counts
rownames(157600): chr1-10001196-10001745 chr1-100027917-100029695 ...
  chrX-989754-991253 chrX-9995827-9996208
rowData names(2): PeakName bias
colnames(7821): K562_Xu_AAACAGCCAAGCGATG-1 K562_Xu_AAACAGCCACATGCTA-1
  ... K562_Xu_TTTGTTGGTTTATGGG-1 K562_Xu_TTTGTTGGTTTGTGGA-1
colData names(0):

In [9]:
bg <- chromVAR::getBackgroundPeaks(ATAC.se, niterations = 100)

In [10]:
index.gene = data.frame(gene_name = rownames(matrix.rna),
                       index = 1:nrow(matrix.rna))
rownames(index.gene) = index.gene$gene_name

index.peak = data.frame(peak_name = rownames(matrix.atac),
                       index = 1:nrow(matrix.atac))
rownames(index.peak) = index.peak$peak_name

genePeakOv = data.frame(queryHits = index.gene[pairs.E2G.filter$TargetGene, "index"],
                        subjectHits = index.peak[pairs.E2G.filter$PeakName, "index"])
genePeakOv

queryHits,subjectHits
<int>,<int>
255,1
412,1
413,1
275,1
382,1
375,1
246,1
416,1
348,1


Run FigR prediction

In [11]:
start_time <- Sys.time()
ObsCor = PeakGeneCor(ATAC = matrix.atac, 
                     RNA = matrix.rna,
                     OV = genePeakOv, 
                     chunkSize = 500,
                     ncores = 16, 
                     bg = bg)
end_time <- Sys.time()
execution_time <- end_time - start_time
execution_time
ObsCor

Running in parallel using  16 cores ..
Computing observed correlations ..
Finished!

Time Elapsed:  0.53041672706604 secs 

Computing background correlations ..


Loading required package: parallel




Time Elapsed:  46.5783624649048 secs 



Time difference of 52.77343 secs

Gene,Peak,rObs,rBg1,rBg2,rBg3,rBg4,rBg5,rBg6,rBg7,⋯,rBg91,rBg92,rBg93,rBg94,rBg95,rBg96,rBg97,rBg98,rBg99,rBg100
<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
255,1,0.008716981,0.0013836384,0.001157918,0.021546763,0.006783589,-0.003833827,0.005257450,0.005506933,⋯,-0.005532437,0.02528881,0.003692097,0.013426458,0.0011904373,0.008533105,0.017175366,0.011343696,-0.002991538,0.005720513
412,1,0.027548958,0.0004732876,-0.002394844,-0.009721982,-0.002322292,0.009618580,0.005583602,0.002663803,⋯,-0.002010846,-0.01242037,0.017861699,-0.003506205,-0.0004121362,0.010733493,-0.009648221,0.008879961,0.011025421,0.007243026
413,1,0.008716981,0.0013836384,0.001157918,0.021546763,0.006783589,-0.003833827,0.005257450,0.005506933,⋯,-0.005532437,0.02528881,0.003692097,0.013426458,0.0011904373,0.008533105,0.017175366,0.011343696,-0.002991538,0.005720513
275,1,0.027548958,0.0004732876,-0.002394844,-0.009721982,-0.002322292,0.009618580,0.005583602,0.002663803,⋯,-0.002010846,-0.01242037,0.017861699,-0.003506205,-0.0004121362,0.010733493,-0.009648221,0.008879961,0.011025421,0.007243026
382,1,0.008716981,0.0013836384,0.001157918,0.021546763,0.006783589,-0.003833827,0.005257450,0.005506933,⋯,-0.005532437,0.02528881,0.003692097,0.013426458,0.0011904373,0.008533105,0.017175366,0.011343696,-0.002991538,0.005720513
375,1,0.027548958,0.0004732876,-0.002394844,-0.009721982,-0.002322292,0.009618580,0.005583602,0.002663803,⋯,-0.002010846,-0.01242037,0.017861699,-0.003506205,-0.0004121362,0.010733493,-0.009648221,0.008879961,0.011025421,0.007243026
246,1,0.008716981,0.0013836384,0.001157918,0.021546763,0.006783589,-0.003833827,0.005257450,0.005506933,⋯,-0.005532437,0.02528881,0.003692097,0.013426458,0.0011904373,0.008533105,0.017175366,0.011343696,-0.002991538,0.005720513
416,1,0.027548958,0.0004732876,-0.002394844,-0.009721982,-0.002322292,0.009618580,0.005583602,0.002663803,⋯,-0.002010846,-0.01242037,0.017861699,-0.003506205,-0.0004121362,0.010733493,-0.009648221,0.008879961,0.011025421,0.007243026
348,1,0.008716981,0.0013836384,0.001157918,0.021546763,0.006783589,-0.003833827,0.005257450,0.005506933,⋯,-0.005532437,0.02528881,0.003692097,0.013426458,0.0011904373,0.008533105,0.017175366,0.011343696,-0.002991538,0.005720513


In [12]:
pairs.E2G.res = pairs.E2G.filter
pairs.E2G.res$rObs = ObsCor[,"rObs"]
pairs.E2G.res$rBgSD <- matrixStats::rowSds(as.matrix(ObsCor[, 4:103]))
pairs.E2G.res$rBgMean <- rowMeans(ObsCor[, 4:103])
pairs.E2G.res$pvalZ <- 1 - stats::pnorm(q = pairs.E2G.res$rObs, 
                                        mean = pairs.E2G.res$rBgMean,
                                        sd = pairs.E2G.res$rBgSD)
pairs.E2G.res

GRanges object with 11201948 ranges and 7 metadata columns:
             seqnames            ranges strand |  TargetGene
                <Rle>         <IRanges>  <Rle> | <character>
         [1]     chr1 10001196-10001745      * |       ACOT7
         [2]     chr1 10001196-10001745      * |      AGTRAP
         [3]     chr1 10001196-10001745      * |    C1orf167
         [4]     chr1 10001196-10001745      * |      CAMTA1
         [5]     chr1 10001196-10001745      * |       CASZ1
         ...      ...               ...    ... .         ...
  [11201944]     chrX   9995827-9996208      * |       TBL1X
  [11201945]     chrX   9995827-9996208      * |      TCEANC
  [11201946]     chrX   9995827-9996208      * |      TMSB4X
  [11201947]     chrX   9995827-9996208      * |     TRAPPC2
  [11201948]     chrX   9995827-9996208      * |        WWC3
                           PeakName               PairName       rObs     rBgSD
                        <character>            <character>  <numeri

Save results

In [13]:
dir.create(dir.output,recursive = T)
saveRDS(pairs.E2G.res,
        paste(dir.output,"pairs.E2G.res.rds",sep = "/"))
pairs.E2G.res

GRanges object with 11201948 ranges and 7 metadata columns:
             seqnames            ranges strand |  TargetGene
                <Rle>         <IRanges>  <Rle> | <character>
         [1]     chr1 10001196-10001745      * |       ACOT7
         [2]     chr1 10001196-10001745      * |      AGTRAP
         [3]     chr1 10001196-10001745      * |    C1orf167
         [4]     chr1 10001196-10001745      * |      CAMTA1
         [5]     chr1 10001196-10001745      * |       CASZ1
         ...      ...               ...    ... .         ...
  [11201944]     chrX   9995827-9996208      * |       TBL1X
  [11201945]     chrX   9995827-9996208      * |      TCEANC
  [11201946]     chrX   9995827-9996208      * |      TMSB4X
  [11201947]     chrX   9995827-9996208      * |     TRAPPC2
  [11201948]     chrX   9995827-9996208      * |        WWC3
                           PeakName               PairName       rObs     rBgSD
                        <character>            <character>  <numeri

In [14]:
df.output = as.data.frame(pairs.E2G.res)
colnames(df.output)[1] = "chr"
df.output[,"CellType"] = "K562"
data.table::fwrite(df.output,
                   file = paste(dir.output,"pairs.E2G.res.tsv.gz",sep = "/"),
                   row.names = F,
                   quote = F,
                   sep = "\t")
df.output

chr,start,end,width,strand,TargetGene,PeakName,PairName,rObs,rBgSD,rBgMean,pvalZ,CellType
<fct>,<int>,<int>,<int>,<fct>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
chr1,10001196,10001745,550,*,ACOT7,chr1-10001196-10001745,chr1-10001196-10001745_ACOT7,0.008716981,0.01087734,0.004654715,0.35440251,K562
chr1,10001196,10001745,550,*,AGTRAP,chr1-10001196-10001745,chr1-10001196-10001745_AGTRAP,0.027548958,0.01062203,0.003689840,0.01234588,K562
chr1,10001196,10001745,550,*,C1orf167,chr1-10001196-10001745,chr1-10001196-10001745_C1orf167,0.008716981,0.01087734,0.004654715,0.35440251,K562
chr1,10001196,10001745,550,*,CAMTA1,chr1-10001196-10001745,chr1-10001196-10001745_CAMTA1,0.027548958,0.01062203,0.003689840,0.01234588,K562
chr1,10001196,10001745,550,*,CASZ1,chr1-10001196-10001745,chr1-10001196-10001745_CASZ1,0.008716981,0.01087734,0.004654715,0.35440251,K562
chr1,10001196,10001745,550,*,CENPS,chr1-10001196-10001745,chr1-10001196-10001745_CENPS,0.027548958,0.01062203,0.003689840,0.01234588,K562
chr1,10001196,10001745,550,*,CHD5,chr1-10001196-10001745,chr1-10001196-10001745_CHD5,0.008716981,0.01087734,0.004654715,0.35440251,K562
chr1,10001196,10001745,550,*,CLCN6,chr1-10001196-10001745,chr1-10001196-10001745_CLCN6,0.027548958,0.01062203,0.003689840,0.01234588,K562
chr1,10001196,10001745,550,*,CLSTN1,chr1-10001196-10001745,chr1-10001196-10001745_CLSTN1,0.008716981,0.01087734,0.004654715,0.35440251,K562


In [15]:
sessionInfo()

R version 4.3.3 (2024-02-29)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Red Hat Enterprise Linux 8.9 (Ootpa)

Matrix products: default
BLAS/LAPACK: /maps/projects/ralab/people/lpm537/software/anaconda3/envs/Notebook_E2G_240505/lib/libopenblasp-r0.3.27.so;  LAPACK version 3.12.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Europe/Copenhagen
tzcode source: system (glibc)

attached base packages:
 [1] parallel  grid      stats4    stats     graphics  grDevices utils    
 [8] datasets  methods   base     

other attached packages:
 [1] pbmcapply_1.5.1                   foreach_1.5.2                    
 [3] BSgenome.Hsapiens.UCSC.hg38_1.4.5 BSgenome_1.